In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2

# This is needed to add the repo dir to the path so jupyter
# can load the modules in the scripts directory from the notebooks
import os
import sys
repo_dir = os.path.split(os.getcwd())[0]
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)
    
import numpy as np
import pandas as pd
import json
import csv
from collections import Counter
from elasticsearch import Elasticsearch
import gzip
import os

data_dir = '/Volumes/Samsung_T5/Data/Book-Reviews/GoodReads/'

author_file = os.path.join(data_dir, 'goodreads_book_authors.json.gz') # author information
book_file = os.path.join(data_dir, 'goodreads_books.json.gz') # basic book metadata
genre_file = os.path.join(data_dir, 'goodreads_book_genres_initial.json.gz') # book genre information
review_text_file = os.path.join(data_dir, 'goodreads_reviews_dedup.json.gz') # includes text



es = Elasticsearch()

/Users/marijnkoolen/Code/Huygens/scale


In [21]:
from helper import read_json

author_name = {}
for record in read_json(author_file):
    author_name[record['author_id']] = record['name']


In [22]:
book_genre = {}
for record in read_json(genre_file):
    book_genre[record['book_id']] = record['genres']


In [34]:
from helper import read_csv

book_meta = {}
#for ri, record in enumerate(read_csv(book_file)):
#    book_meta[record['book_id']] = record
#    if (ri+1) % 1000000 == 0:
#        print(ri+1, 'records read')
        
#print(ri+1, 'records read')

book_meta = {}
for ri, record in enumerate(read_json(book_file)):
    book_meta[record['book_id']] = record
    del record['popular_shelves']
    del record['description']
    for author_info in record['authors']:
        if author_info['author_id'] in author_name:
            author_info['author_name'] = author_name[author_info['author_id']]
    if record['book_id'] in book_genre:
        record['genres'] = book_genre[record['book_id']]
    else:
        record['genres'] = None
    if (ri+1) % 100000 == 0:
        print(ri+1, 'records read')

print(ri+1, 'records read')


100000 records read
200000 records read
300000 records read
400000 records read
500000 records read
600000 records read
700000 records read
800000 records read
900000 records read
1000000 records read
1100000 records read
1200000 records read
1300000 records read
1400000 records read
1500000 records read
1600000 records read
1700000 records read
1800000 records read
1900000 records read
2000000 records read
2100000 records read
2200000 records read
2300000 records read
2360655 records read


In [41]:
import time
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from elasticsearch.helpers import bulk

def make_book_query(book_id):
    return {
        'query': {
            'match': {
                'book_id': book_id
            }
        }
    }

def get_book_record(book_id):
    query = make_book_query(book_id)
    if es.exists(index=book_index, id=book_id):
        response = es.get(index=book_index, id=book_id)
        return response['_source']
    else:
        raise ValueError(f'unknown book record: {book_id}')
        return {}

def make_transaction(review):    
    return {'_index': review_index, '_doc_type': 'review', '_id': review['review_id'], '_source': review}

def make_transactions(reviews):
    return [make_transaction(review) for review in reviews]
        
def index_reviews(reviews):
    transactions = make_transactions(reviews)
    bulk(es, transactions)

book_index = 'goodreads-ucsd-books'
review_index = 'goodreads-ucsd-reviews'

#if es.indices.exists(index=review_index):
#    es.indices.delete(index=review_index)


reviews = []

start = time.time()
if es.indices.exists(index=review_index):
    query = {'size': 0, 'track_total_hits': True}
    response = es.search(index=review_index, body=query)
    reviews_indexed = response['hits']['total']['value']
else:
    reviews_indexed = 0
    
print(f'Starting with {reviews_indexed} reviews indexed')

for ri, review in enumerate(read_json(review_text_file)):
    if (ri+1) < reviews_indexed:
        continue
    #book_record = get_book_record(review['book_id'])
    book_record = book_meta[review['book_id']]
    try:
        review['review_lang'] = detect(review['review_text'])
    except LangDetectException:
        review['review_lang'] = 'unknown'
    for field in ['authors', 'genres', 'work_id', 'title', 'title_without_series', 'language_code']:
        review[field] = book_record[field]
    reviews += [review]
    if len(reviews) == 200:
        index_reviews(reviews)
        reviews = []
    if (ri+1) % 10000 == 0:
        print(ri+1, 'reviews indexed', time.time() - start, 'seconds')

    
index_reviews(reviews)



Starting with 8437800 reviews indexed
8440000 reviews indexed 150.23579216003418 seconds
8450000 reviews indexed 221.94548296928406 seconds
8460000 reviews indexed 314.18526816368103 seconds
8470000 reviews indexed 377.2259192466736 seconds
8480000 reviews indexed 440.39561009407043 seconds
8490000 reviews indexed 503.22113704681396 seconds
8500000 reviews indexed 577.2425200939178 seconds
8510000 reviews indexed 656.7471890449524 seconds
8520000 reviews indexed 718.4459512233734 seconds
8530000 reviews indexed 781.2018342018127 seconds
8540000 reviews indexed 851.2586460113525 seconds
8550000 reviews indexed 915.4782891273499 seconds
8560000 reviews indexed 980.6604719161987 seconds
8570000 reviews indexed 1049.2800810337067 seconds
8580000 reviews indexed 1117.1197609901428 seconds
8590000 reviews indexed 1176.1500480175018 seconds
8600000 reviews indexed 1259.8222739696503 seconds
8610000 reviews indexed 1323.0146460533142 seconds
8620000 reviews indexed 1384.6933929920197 seconds
8

10060000 reviews indexed 12415.22952914238 seconds
10070000 reviews indexed 12504.123789072037 seconds
10080000 reviews indexed 12578.645013093948 seconds
10090000 reviews indexed 12641.378710985184 seconds
10100000 reviews indexed 12714.986653089523 seconds
10110000 reviews indexed 12776.05956697464 seconds
10120000 reviews indexed 12835.491581201553 seconds
10130000 reviews indexed 12901.927097082138 seconds
10140000 reviews indexed 12970.306122303009 seconds
10150000 reviews indexed 13038.13146686554 seconds
10160000 reviews indexed 13112.838850975037 seconds
10170000 reviews indexed 13179.843383073807 seconds
10180000 reviews indexed 13250.534442186356 seconds
10190000 reviews indexed 13334.793855905533 seconds
10200000 reviews indexed 13411.671495199203 seconds
10210000 reviews indexed 13476.654930114746 seconds
10220000 reviews indexed 13548.393862009048 seconds
10230000 reviews indexed 13612.29379606247 seconds
10240000 reviews indexed 13681.943649053574 seconds
10250000 reviews

11650000 reviews indexed 23904.636642932892 seconds
11660000 reviews indexed 23971.057899951935 seconds
11670000 reviews indexed 24035.479831933975 seconds
11680000 reviews indexed 24109.577673196793 seconds
11690000 reviews indexed 24187.017925024033 seconds
11700000 reviews indexed 24255.44843196869 seconds
11710000 reviews indexed 24322.872616052628 seconds
11720000 reviews indexed 24407.04327106476 seconds
11730000 reviews indexed 24473.61480998993 seconds
11740000 reviews indexed 24547.036448955536 seconds
11750000 reviews indexed 24617.94363617897 seconds
11760000 reviews indexed 24682.277657985687 seconds
11770000 reviews indexed 24745.564383268356 seconds
11780000 reviews indexed 24834.618134975433 seconds
11790000 reviews indexed 24908.719053030014 seconds
11800000 reviews indexed 24969.221306085587 seconds
11810000 reviews indexed 25037.90005493164 seconds
11820000 reviews indexed 25113.308714151382 seconds
11830000 reviews indexed 25181.112746953964 seconds
11840000 reviews 

13240000 reviews indexed 35268.06675410271 seconds
13250000 reviews indexed 35353.87786912918 seconds
13260000 reviews indexed 35443.8358631134 seconds
13270000 reviews indexed 35525.10008311272 seconds
13280000 reviews indexed 35606.20055413246 seconds
13290000 reviews indexed 35690.81394410133 seconds
13300000 reviews indexed 35767.01699399948 seconds
13310000 reviews indexed 35832.58572411537 seconds
13320000 reviews indexed 35913.91709303856 seconds
13330000 reviews indexed 35978.528945207596 seconds
13340000 reviews indexed 36045.18008995056 seconds
13350000 reviews indexed 36116.929486989975 seconds
13360000 reviews indexed 36194.42374610901 seconds
13370000 reviews indexed 36268.15837812424 seconds
13380000 reviews indexed 36357.17974591255 seconds
13390000 reviews indexed 36429.528405189514 seconds
13400000 reviews indexed 36501.96964216232 seconds
13410000 reviews indexed 36579.73637008667 seconds
13420000 reviews indexed 36645.935779094696 seconds
13430000 reviews indexed 367

14850000 reviews indexed 45959.747534036636 seconds
14860000 reviews indexed 46028.50758910179 seconds
14870000 reviews indexed 46093.93666911125 seconds
14880000 reviews indexed 46155.309712171555 seconds
14890000 reviews indexed 46207.6284263134 seconds
14900000 reviews indexed 46270.488672971725 seconds
14910000 reviews indexed 46323.82267212868 seconds
14920000 reviews indexed 46390.25274705887 seconds
14930000 reviews indexed 46453.91786527634 seconds
14940000 reviews indexed 46510.78788805008 seconds
14950000 reviews indexed 46568.30861091614 seconds
14960000 reviews indexed 46623.330637931824 seconds
14970000 reviews indexed 46680.80394697189 seconds
14980000 reviews indexed 46737.26415395737 seconds
14990000 reviews indexed 46792.57344508171 seconds
15000000 reviews indexed 46848.62968206406 seconds
15010000 reviews indexed 46918.06412196159 seconds
15020000 reviews indexed 46981.22699213028 seconds
15030000 reviews indexed 47040.574971199036 seconds
15040000 reviews indexed 47

## The CSV files

In [ ]:
data_dir = '/Volumes/Samsung_T5/Data/Book-Reviews/GoodReads/'

author_file = os.path.join(data_dir, 'goodreads_book_authors.csv.gz') # author information
book_file = os.path.join(data_dir, 'goodreads_books.csv.gz') # basic book metadata
genre_file = os.path.join(data_dir, 'goodreads_book_genres_initial.csv.gz') # book genre information
review_file = os.path.join(data_dir, 'goodreads_reviews_dedup-no_text.csv.gz') # excludes text to save memory
review_text_file = os.path.join(data_dir, 'goodreads_reviews_dedup.csv.gz') # includes text
review_filtered_file = os.path.join(data_dir, 'goodreads_reviews_dedup_filtered-no_text.csv.gz') # excludes text and non-reviews



In [30]:
from helper import read_csv

author_name = {}
for record in read_csv(author_file):
    author_name[record['author_id']] = record['name']


In [31]:
book_genre = {}
for record in read_csv(genre_file):
    book_genre[record['book_id']] = record['genres']


In [34]:
book_meta = {}
for record in read_csv(book_file):
    book_meta[record['book_id']] = record
    record['author_name'] = author_name[record['author_id']]
    if record['book_id'] in book_genre:
        record['genres'] = book_genre[record['book_id']]
    else:
        record['genres'] = None


In [3]:
import spacy
from helper import scroll_hits
from elasticsearch.helpers import bulk

def make_token(token):
    return {'word': token.text, 'lemma': token.lemma_, 'pos': token.pos_}

def make_sent_hit(sent, review, num_sents):
    tokens = [make_token(token) for token in sent]
    sent_doc = {
        'sent_id': f'{review["review_id"]}-{num_sents}',
        'review_id': review['review_id'],
        'user_id': review['user_id'],
        'book_id': review['book_id'],
        'work_id': review['work_id'],
        'num_chars': len(sent.text),
        'num_words': len(sent),
        'sentence_text': sent.text,
        'tokens': tokens,
        'review_lang': review['review_lang']
    }
    return {'_index': sent_index, '_type': 'sentence', '_id': sent_doc['sent_id'], '_source':sent_doc}

def index_sentences(query, nlp, start_hi=0):
    sent_count = 0
    review_hits = []
    sent_hits = []
    for hi, hit in enumerate(scroll_hits(es, query, index=review_index, size=100)):
        if hi < start_hi:
            continue
        review = hit['_source']
        spacy_doc = nlp(review['review_text'])
        review['num_chars'] = len(review['review_text'])
        review['num_words'] = len(spacy_doc)
        num_sents = 0
        for sent in spacy_doc.sents:
            num_sents += 1
            sent_hit = make_sent_hit(sent, review, num_sents)
            sent_hits += [sent_hit]
        review['num_sents'] = num_sents
        sent_count += num_sents
        review_hits += [hit]
        if (hi+1) % 10 == 0:
            bulk(es, sent_hits)
            bulk(es, review_hits)
            review_hits = []
            sent_hits = []
        if (hi+1) % 100000 == 0:
            print(hi+1, 'reviews re-indexed\t', sent_count, 'sentences indexed')
    print(hi+1, 'reviews re-indexed\t', sent_count, 'sentences indexed')



In [46]:
#nlp_nl = spacy.load('nl_core_news_lg')

review_index = 'goodreads-ucsd-reviews'
sent_index = 'goodreads-ucsd-review-sents'

query = {
    'query': {
        'match': {
            'review_lang': 'nl'
        }
    }
}

index_sentences(query, nlp_nl)


total hits: {'value': 45973, 'relation': 'eq'}
10000 reviews re-indexed	 61566 sentences indexed
20000 reviews re-indexed	 138341 sentences indexed
30000 reviews re-indexed	 213826 sentences indexed
40000 reviews re-indexed	 284985 sentences indexed
45973 reviews re-indexed	 319413 sentences indexed


In [ ]:
nlp_en = spacy.load("en_core_web_lg")

query = {
    'query': {
        'match': {
            'review_lang': 'en'
        }
    }
}

review_index = 'goodreads-ucsd-reviews'
sent_index = 'goodreads-ucsd-review-sents'


start_hi = 0
index_sentences(query, nlp_en, start_hi=start_hi)


total hits: {'value': 13896258, 'relation': 'eq'}
10000 reviews re-indexed	 88559 sentences indexed
20000 reviews re-indexed	 198091 sentences indexed
30000 reviews re-indexed	 312657 sentences indexed
40000 reviews re-indexed	 400940 sentences indexed
50000 reviews re-indexed	 495624 sentences indexed
60000 reviews re-indexed	 614312 sentences indexed
70000 reviews re-indexed	 699152 sentences indexed
80000 reviews re-indexed	 787343 sentences indexed
90000 reviews re-indexed	 873289 sentences indexed
100000 reviews re-indexed	 995384 sentences indexed
110000 reviews re-indexed	 1093568 sentences indexed
120000 reviews re-indexed	 1200109 sentences indexed
130000 reviews re-indexed	 1271175 sentences indexed
